<a href="https://colab.research.google.com/github/jjgl0540-hub/Leeds_Energy_Project_GEOG5415M/blob/main/GEOG5415M_Final_Project_01_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEOG5415M Final Assignment (Template)

Student ID number:202018307

In [ ]:
# read in required packages

# import all packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import geopandas as gpd

In [ ]:
# configure plotting settings
%matplotlib inline
sns.set_theme(style="whitegrid")

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This is an example block of mardown text I want to reference <a href="#ref1">[1]</a>. I might need to add some more citations <a href="#ref2">[2]</a><a href="#ref2">[3]</a>



In [ ]:
# load the data
epc=pd.read_csv('/content/drive/MyDrive/Leeds_Energy_Project_GEOG5415M/data/raw/epc_leeds_2025.csv')
imd=pd.read_csv('/content/drive/MyDrive/Leeds_Energy_Project_GEOG5415M/data/raw/imd_2019.csv')
lsoa=gpd.read_file('/content/drive/MyDrive/Leeds_Energy_Project_GEOG5415M/data/raw/LSOA/LSOA_2021_EW_BGC_V5.shp')


/tmp/ipython-input-2001626475.py:2: DtypeWarning: Columns (15,37,39,40,83) have mixed types. Specify dtype option on import or set low_memory=False.
  epc=pd.read_csv('/content/drive/MyDrive/Leeds_Energy_Project_GEOG5415M/data/raw/epc_leeds_2025.csv')


In [ ]:
# load remaining data
# define a helper function to read the NOMIS census CSV file.
# the top of the NOMIS file contains several lines of metadata,
# therefore, the first 7 lines are skipped to read the header correctly.
def read_nomis_csv(path, skiprows=7):
    return pd.read_csv(path, skiprows=skiprows)

heating=read_nomis_csv('/content/drive/MyDrive/Leeds_Energy_Project_GEOG5415M/data/raw/census_heating_2021.csv')
students=read_nomis_csv('/content/drive/MyDrive/Leeds_Energy_Project_GEOG5415M/data/raw/census_students_2021.csv')
tenure=read_nomis_csv('/content/drive/MyDrive/Leeds_Energy_Project_GEOG5415M/data/raw/census_tenure_2021.csv')

In [ ]:
# after loading each dataset, basic data integrity checks (such as structure and completeness) were performed.
# this demonstration shows the checks performed on the EPC dataset.

epc.head()

,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,...,CONSTITUENCY_LABEL,POSTTOWN,CONSTRUCTION_AGE_BAND,LODGEMENT_DATETIME,TENURE,FIXED_LIGHTING_OUTLETS_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,UPRN,UPRN_SOURCE,REPORT_TYPE
0,001dca82162cbdf798f746f53027c0cc82afda971d5e84...,Flat 2,35a Town Street,Farsley,LS28 5HX,10003805692,E,C,42,70,...,Pudsey,PUDSEY,England and Wales: before 1900,2022-12-06 16:54:30,Rented (private),9.0,NaN,72736113.0,Energy Assessor,100
1,18a413eb393a87aaccdcbbc35d75c519a13bf9713e5d8d...,8 Sackville Street,NaN,NaN,LS7 2AS,10003489215,D,B,61,82,...,Leeds North East,LEEDS,England and Wales: 1900-1929,2022-10-03 15:40:47,Rented (private),12.0,NaN,72658276.0,Energy Assessor,100
2,18ad988b1d1a739e5adfd04531c27c63e7ea810f40c31d...,20 Highfield Gardens,NaN,NaN,LS12 4DU,10003423919,C,B,72,86,...,Leeds West,LEEDS,England and Wales: 2007-2011,2022-10-25 07:49:22,Rented (social),6.0,NaN,72683080.0,Energy Assessor,100
3,18bd4432597b334e2d8031b1ded0e4d713f2133e939b89...,17 Chestnut Gardens,NaN,NaN,LS12 4LP,10002989904,B,A,88,92,...,Leeds West,Leeds,England and Wales: 1983-1990,2022-10-06 16:09:42,Owner-occupied,7.0,NaN,72266470.0,Address Matched,100
4,275687798052018050310193492080467,"9, Foxwood Farm Way",NaN,NaN,LS8 3EE,4759821668,D,B,58,87,...,Leeds East,LEEDS,England and Wales: 1983-1990,2018-05-03 10:19:34,owner-occupied,NaN,NaN,72269527.0,Address Matched,100


In [54]:
# 1.Data cleaning
# 1.1 uniform key column naming
heating=heating.rename(columns={'mnemonic': 'lsoa_code'})
students=students.rename(columns={'mnemonic': 'lsoa_code'})
tenure=tenure.rename(columns={'mnemonic': 'lsoa_code'})

In [55]:
# 1.2 combine a census table
lsoa_census = (heating.merge(students, on='lsoa_code', how='left').merge(tenure, on='lsoa_code', how='left'))

In [ ]:
# 1.3 process LSOA datasets

# uniform key column naming and screen out Leeds' LSOA
lsoa=lsoa.rename(columns={'LSOA21CD': 'lsoa_code'})
lsoa_leeds=lsoa[lsoa['lsoa_code'].isin(lsoa_census['lsoa_code'])].copy()

# paste the census attributes.
lsoa_geo=lsoa_leeds.merge(lsoa_census, on='lsoa_code', how='left')

In [ ]:
# 1.4 process epc dataset


,2021 super output area - lower layer_x,lsoa_code,Total: All households_x,No central heating,Mains gas only,Tank or bottled gas only,Electric only,Oil only,Wood only,Solid fuel only,...,Not a student,2021 super output area - lower layer,Total: All households_y,Owned,Shared ownership,Social rented,Private rented,Lives rent free,Owns with a mortgage or loan or shared ownership,Private rented or lives rent free
0,Leeds 001A,E01011698,633.0,7.0,565.0,2.0,11.0,2.0,0.0,0.0,...,1146.0,Leeds 001A,632.0,518.0,1.0,55.0,57.0,1.0,226.0,58.0
1,Leeds 001B,E01011699,578.0,3.0,536.0,0.0,5.0,1.0,0.0,0.0,...,1051.0,Leeds 001B,576.0,506.0,1.0,5.0,64.0,0.0,197.0,64.0
2,Leeds 001C,E01011701,596.0,5.0,542.0,1.0,6.0,0.0,0.0,0.0,...,1082.0,Leeds 001C,597.0,503.0,8.0,36.0,49.0,1.0,196.0,50.0
3,Leeds 001D,E01011702,665.0,0.0,587.0,3.0,21.0,0.0,0.0,1.0,...,1257.0,Leeds 001D,665.0,605.0,4.0,14.0,42.0,0.0,267.0,42.0
4,Leeds 001E,E01011703,567.0,0.0,501.0,0.0,31.0,0.0,2.0,0.0,...,1055.0,Leeds 001E,565.0,539.0,0.0,7.0,19.0,0.0,133.0,19.0


In [ ]:
2.先把 EPC 聚合到 LSOA 层级，再 merge

In [ ]:
tenure.head()

,2021 super output area - lower layer,mnemonic,Total: All households,Owned,Shared ownership,Social rented,Private rented,Lives rent free,Owns with a mortgage or loan or shared ownership,Private rented or lives rent free
0,Leeds 001A,E01011698,632.0,518.0,1.0,55.0,57.0,1.0,226.0,58.0
1,Leeds 001B,E01011699,576.0,506.0,1.0,5.0,64.0,0.0,197.0,64.0
2,Leeds 001C,E01011701,597.0,503.0,8.0,36.0,49.0,1.0,196.0,50.0
3,Leeds 001D,E01011702,665.0,605.0,4.0,14.0,42.0,0.0,267.0,42.0
4,Leeds 001E,E01011703,565.0,539.0,0.0,7.0,19.0,0.0,133.0,19.0


In [ ]:
heating.head()

,2021 super output area - lower layer,mnemonic,Total: All households,No central heating,Mains gas only,Tank or bottled gas only,Electric only,Oil only,Wood only,Solid fuel only,Renewable energy only,District or communal heat networks only,Other central heating only,Two or more types of central heating (not including renewable energy),Two or more types of central heating (including renewable energy)
0,Leeds 001A,E01011698,633.0,7.0,565.0,2.0,11.0,2.0,0.0,0.0,0.0,0.0,2.0,42.0,2.0
1,Leeds 001B,E01011699,578.0,3.0,536.0,0.0,5.0,1.0,0.0,0.0,1.0,0.0,0.0,29.0,3.0
2,Leeds 001C,E01011701,596.0,5.0,542.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,4.0
3,Leeds 001D,E01011702,665.0,0.0,587.0,3.0,21.0,0.0,0.0,1.0,2.0,0.0,0.0,41.0,10.0
4,Leeds 001E,E01011703,567.0,0.0,501.0,0.0,31.0,0.0,2.0,0.0,0.0,2.0,0.0,30.0,1.0


In [ ]:
students.head()

,2021 super output area - lower layer,mnemonic,Total,Student,Not a student
0,Leeds 001A,E01011698,1400.0,254.0,1146.0
1,Leeds 001B,E01011699,1226.0,175.0,1051.0
2,Leeds 001C,E01011701,1300.0,218.0,1082.0
3,Leeds 001D,E01011702,1548.0,291.0,1257.0
4,Leeds 001E,E01011703,1198.0,143.0,1055.0


In [ ]:
imd.head()

,LSOA code (2011),LSOA name (2011),Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Income Rank (where 1 is most deprived),Income Decile (where 1 is most deprived 10% of LSOAs),...,Indoors Sub-domain Rank (where 1 is most deprived),Indoors Sub-domain Decile (where 1 is most deprived 10% of LSOAs),Outdoors Sub-domain Score,Outdoors Sub-domain Rank (where 1 is most deprived),Outdoors Sub-domain Decile (where 1 is most deprived 10% of LSOAs),Total population: mid 2015 (excluding prisoners),Dependent Children aged 0-15: mid 2015 (excluding prisoners),Population aged 16-59: mid 2015 (excluding prisoners),Older population aged 60 and over: mid 2015 (excluding prisoners),Working age population 18-59/64: for use with Employment Deprivation Domain (excluding prisoners)
0,E01000001,City of London 001A,E09000001,City of London,6.208,29199,9,0.007,32831,10,...,16364,5,1.503,1615,1,1296,175,656,465,715
1,E01000002,City of London 001B,E09000001,City of London,5.143,30379,10,0.034,29901,10,...,22676,7,1.196,2969,1,1156,182,580,394,620
2,E01000003,City of London 001C,E09000001,City of London,19.402,14915,5,0.086,18510,6,...,17318,6,2.207,162,1,1350,146,759,445,804
3,E01000005,City of London 001E,E09000001,City of London,28.652,8678,3,0.211,6029,2,...,25218,8,1.769,849,1,1121,229,692,200,683
4,E01000006,Barking and Dagenham 016A,E09000002,Barking and Dagenham,19.837,14486,5,0.117,14023,5,...,14745,5,0.969,4368,2,2040,522,1297,221,1285


In [ ]:
lsoa.head()

,LSOA21CD,LSOA21NM,LSOA21NMW,BNG_E,BNG_N,LAT,LONG,GlobalID,geometry
0,E01000001,City of London 001A,None,532123,181632,51.5182,-0.097150,86214465-5cf4-4e8f-9492-3667471c42d6,"POLYGON ((532105.312 182010.574, 532162.491 18..."
1,E01000002,City of London 001B,None,532480,181715,51.5188,-0.091970,cd40c491-6567-405f-8c18-426e17b356ce,"POLYGON ((532634.497 181926.016, 532619.141 18..."
2,E01000003,City of London 001C,None,532239,182033,51.5217,-0.095330,7fd27aaf-d858-4e46-9099-92b43f66b948,"POLYGON ((532135.138 182198.131, 532158.25 182..."
3,E01000005,City of London 001E,None,533581,181283,51.5147,-0.076280,7e76a16a-028f-4f49-84b5-6e5a67322f3c,"POLYGON ((533808.018 180767.774, 533649.037 18..."
4,E01000006,Barking and Dagenham 016A,None,544994,184274,51.5387,0.089317,25ab047e-6fcf-4f76-9176-e92e44c0e097,"POLYGON ((545122.049 184314.931, 545271.849 18..."


In [ ]:
#try:
  epc = pd.read_csv('/content/drive/MyDrive/Leeds_Energy_Project_GEOG5415M/data/raw/epc_leeds_2025.csv')


  print("Success! Data loaded.")
  print(f"The dataset has {epc.shape[0]} rows and {epc.shape[1]} columns.")

  display(epc.head())

#except FileNotFoundError:
  print("Error: File not found!")
  print("Please check: 1. Did you upload the file? 2. Is the filename correct?")

/tmp/ipython-input-215272112.py:2: DtypeWarning: Columns (15,37,39,40,83) have mixed types. Specify dtype option on import or set low_memory=False.
  epc = pd.read_csv('/content/drive/MyDrive/Leeds_Energy_Project_GEOG5415M/data/raw/epc_leeds_2025.csv')


Success! Data loaded.
The dataset has 417714 rows and 93 columns.


,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,...,CONSTITUENCY_LABEL,POSTTOWN,CONSTRUCTION_AGE_BAND,LODGEMENT_DATETIME,TENURE,FIXED_LIGHTING_OUTLETS_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,UPRN,UPRN_SOURCE,REPORT_TYPE
0,001dca82162cbdf798f746f53027c0cc82afda971d5e84...,Flat 2,35a Town Street,Farsley,LS28 5HX,10003805692,E,C,42,70,...,Pudsey,PUDSEY,England and Wales: before 1900,2022-12-06 16:54:30,Rented (private),9.0,NaN,72736113.0,Energy Assessor,100
1,18a413eb393a87aaccdcbbc35d75c519a13bf9713e5d8d...,8 Sackville Street,NaN,NaN,LS7 2AS,10003489215,D,B,61,82,...,Leeds North East,LEEDS,England and Wales: 1900-1929,2022-10-03 15:40:47,Rented (private),12.0,NaN,72658276.0,Energy Assessor,100
2,18ad988b1d1a739e5adfd04531c27c63e7ea810f40c31d...,20 Highfield Gardens,NaN,NaN,LS12 4DU,10003423919,C,B,72,86,...,Leeds West,LEEDS,England and Wales: 2007-2011,2022-10-25 07:49:22,Rented (social),6.0,NaN,72683080.0,Energy Assessor,100
3,18bd4432597b334e2d8031b1ded0e4d713f2133e939b89...,17 Chestnut Gardens,NaN,NaN,LS12 4LP,10002989904,B,A,88,92,...,Leeds West,Leeds,England and Wales: 1983-1990,2022-10-06 16:09:42,Owner-occupied,7.0,NaN,72266470.0,Address Matched,100
4,275687798052018050310193492080467,"9, Foxwood Farm Way",NaN,NaN,LS8 3EE,4759821668,D,B,58,87,...,Leeds East,LEEDS,England and Wales: 1983-1990,2018-05-03 10:19:34,owner-occupied,NaN,NaN,72269527.0,Address Matched,100


Data Visualisation
- Don't forget to present your final two data visualisation (one spatial and one non-spatial) and the justifications about the decisions you made whilst preparing and visualising the data.

## References
<p><a href="add_url_here">[1]</a>Footnote citation goes here</p>
<p><a href="https://github.com/FrancescaPontin/GEOG5415M-Programming-for-Spatial-Data-Science">[2]</a>GEOG5415M course materials</p>
<p><a href="add_url_here">[3]</a>Citation 3</p>